In [2]:
import os
import gym
import gym_donkeycar
import numpy as np
import random
import tensorflow
import shimmy
from rich.live import Live
import stable_baselines3
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import TD3

In [3]:
DONKEY_GYM = True
DONKEY_SIM_PATH = "remote"
DONKEY_GYM_ENV_NAME = "donkey-generated-track-v0"
body_style = "car01"
body_rgb = (128, 128, 128)
car_name = "TD33"
font_size = 100
WEB_CONTROL_PORT = int(os.getenv("WEB_CONTROL_PORT", 8887))
port = 9091

In [4]:
folder = '/mnt/c/Users/user/OneDrive/Desktop/DonkeyCar/TD33'

In [5]:
conf = { "DONKEY_GYM":DONKEY_GYM, "DONKEY_SIM_PATH":DONKEY_SIM_PATH, "DONKEY_GYM_ENV_NAME":DONKEY_GYM_ENV_NAME, 
         "body_style":body_style, "body_rgb":body_rgb, "car_name":car_name, "font_size":font_size, 
         "WEB_CONTROL_PORT" : WEB_CONTROL_PORT, "port" : port }

In [8]:
env = gym.make("donkey-minimonaco-track-v0", conf=conf)

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/gym/spaces/box.py:78: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:sim started!


starting DonkeyGym env
Setting default: start_delay 5.0
Setting default: max_cte 8.0
Setting default: frame_skip 1
Setting default: cam_resolution (120, 160, 3)
Setting default: log_level 20
Setting default: host localhost
Setting default: steer_limit 1.0
Setting default: throttle_min 0.0
Setting default: throttle_max 1.0


INFO:gym_donkeycar.envs.donkey_sim:New lap time: 4.39 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 22.57 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 26.34 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 30.68 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 33.77 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 8.82 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 5.09 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 7.13 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 6.05 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 6.37 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 17.06 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 12.29 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 23.35 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 13.98 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 14.76 seconds
INFO:gym_donkeycar.envs.donkey_sim:New lap time: 21.88 second

In [9]:
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.td3.policies import TD3Policy
import torch 

class CustomTD3Policy(TD3Policy):
    def make_actor_critic_optimizer(self):
        self.actor.optimizer = torch.optim.Adam(self.actor.parameters(), lr=0.0000005 )
        self.critic.optimizer = torch.optim.Adam(self.critic.parameters(), lr=0.000001)

In [10]:
for i in range(1000):
    if i == 0:
        n_actions = env.action_space.shape[-1]
        action_noise = stable_baselines3.common.noise.NormalActionNoise(mean=0.1*np.ones(n_actions), sigma=0.1 * np.ones(n_actions))
        model = TD3(CustomTD3Policy, env, verbose = 0, buffer_size=10000, action_noise = action_noise )
        Log={"TestReward":[]}

    # training
    obs = env.reset()
    model.learn(total_timesteps=2500, progress_bar=0)
    model.save(f'{folder}/{(i+1)*2500}.pth')

    # testing
    tem = 0
    for _ in range(20):
        obs = env.reset()
        while True:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            # print (reward, done)
            env.render()
            tem += reward
            if done: break
    Log["TestReward"].append(tem/20)
    print(f"testing {(i+1)*2500} : {tem/20}")
    np.save(f"{folder}/Log_TD33-{(i+1)*2500}.npy", Log)

/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/stable_baselines3/common/utils.py:488: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  return th.as_tensor(obs, device=device)


testing 2500 : 15.187656265288425
testing 5000 : 14.917359500218804
testing 7500 : 14.695888778824358
testing 10000 : 15.033650641146105
testing 12500 : 15.173929330098545
testing 15000 : 15.00996042778587
testing 17500 : 14.964143304066369
testing 20000 : 15.07057471397604
testing 22500 : 14.897920761234678
testing 25000 : 14.829103073626637
testing 27500 : 15.146632492142022
testing 30000 : 14.74293037104431
testing 32500 : 14.805279757758816
testing 35000 : 15.306853955798434
testing 37500 : 15.04897300140062
testing 40000 : 14.920343049910901
testing 42500 : 15.371799073594469
testing 45000 : 15.473024634171583
testing 47500 : 15.099807051808483
testing 50000 : 14.721648626519572
testing 52500 : 14.920723988866436
testing 55000 : 15.275035143196543
testing 57500 : 13.789993369928576
testing 60000 : 14.531772666070328
testing 62500 : 14.718462023473109
testing 65000 : 14.233821964314634
testing 67500 : 14.959592151150805
testing 70000 : 14.653437297708928
testing 72500 : 14.49883506

KeyboardInterrupt: 

In [11]:
env.close()